In [7]:
# COMP-551 Mini-Project #2.
# Written by Ryan Wilson.
# Encoding = UTF-8.

# Import pandas, numpy and matplotlib.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import preprocessing & classifier modules from scikit-learn.
from sklearn.datasets import fetch_openml
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, auc

In [10]:
#########################################################################
##########          IMPORT & DATAFRAME INITIALIZATION          ##########
#########################################################################

# Load iris dataset from sklearn.
data = fetch_openml(name='wine-quality-white')

# Print full description of dataset.
# pylint: disable=E1101
print(data.DESCR)
print()

# Initialize dataframe & set feature and target arrays to X & Y.
df = pd.DataFrame(data = np.c_[data['data'], data['target']], \
    columns = data['feature_names'] + ['Class'])
#df.to_csv('d:/PhD Thesis/Courses/COMP 551/Python/Python Files/Mini-Project 2/df_wine.csv', index = True)

X = df.iloc[:, :-1].astype(float)
y = df.iloc[:, -1].astype(float)

# Set up print formatting options (adjust significant digits).
np.set_printoptions(formatter={'float': '{:0.4f}'.format})

# Print dataframes to ensure proper import.
print("Feature Data (X):")
print(X)
print()

print("Target Data (y):")
print(y)
print()

Citation Request:
  This dataset is public available for research. The details are described in [Cortez et al., 2009]. 
  Please include this citation if you plan to use this database:

  P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
  Modeling wine preferences by data mining from physicochemical properties.
  In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.

  Available at: [@Elsevier] http://dx.doi.org/10.1016/j.dss.2009.05.016
                [Pre-press (pdf)] http://www3.dsi.uminho.pt/pcortez/winequality09.pdf
                [bib] http://www3.dsi.uminho.pt/pcortez/dss09.bib

1. Title: Wine Quality 

2. Sources
   Created by: Paulo Cortez (Univ. Minho), Antonio Cerdeira, Fernando Almeida, Telmo Matos and Jose Reis (CVRVV) @ 2009
   
3. Past Usage:

  P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
  Modeling wine preferences by data mining from physicochemical properties.
  In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 016

In [11]:

#########################################################################
#################          DATA PREPROCESSING          ##################
#########################################################################

# Mean-center each variable (features & target) by column.
X_mean = X.mean(axis=0)
X_centered = X - X_mean

# Re-scale each variable by setting L2-norm to 1 (column-wise).
X_norm_fn = np.sqrt(np.sum(np.square(X_centered), axis=0))
X_norm = X_centered / X_norm_fn

# Insert leading bias term (w0=1) column to features dataframe.
X_norm = pd.concat([pd.Series(1, index=X_norm.index, name='bias'), X_norm], axis=1)

# One-hot encode multiclass target variable for classification algorithm.
dummies = pd.get_dummies(y[:])
y_encoded = pd.concat([y, dummies], axis=1)
y_encoded = y_encoded.drop(['Class'], axis=1)

#X_norm.to_csv('d:/PhD Thesis/Courses/COMP 551/Python/Python Files/Mini-Project 2/df_wine_X-norm.csv', index = True)
#y_encoded.to_csv('d:/PhD Thesis/Courses/COMP 551/Python/Python Files/Mini-Project 2/df_wine_y-encoded.csv', index = True)

# Preview the preprocessed data to ensure proper scaling.
print("Feature Data X_norm [re-scaled w/ bias term]:")
print(X_norm)
print()

print("Target Data y_encoded [one-hot encoded]:")
print(y_encoded)
print()

# Check for null values (NaNs) generated by preprocessing.
print("X Null Values:", X.isnull().values.any())
print("y Null Values:", y.isnull().values.any())
print()

Feature Data X_norm [re-scaled w/ bias term]:
      bias        V1        V2        V3        V4        V5        V6  \
0        1  0.002459 -0.001168  0.003047  0.040313 -0.000505  0.008144   
1        1 -0.009395  0.003085  0.000686 -0.013499  0.002111 -0.017904   
2        1  0.021086  0.000249  0.007771  0.001433  0.002765 -0.004460   
3        1  0.005846 -0.006839 -0.001676  0.005941  0.007998  0.009824   
4        1  0.005846 -0.006839 -0.001676  0.005941  0.007998  0.009824   
...    ...       ...       ...       ...       ...       ...       ...   
4893     1 -0.011088 -0.009675 -0.005218 -0.013499 -0.004430 -0.009502   
4894     1 -0.004315  0.005920  0.003047  0.004532  0.000803  0.018226   
4895     1 -0.006008 -0.005422 -0.017026 -0.014626 -0.003121 -0.004460   
4896     1 -0.022942  0.001667 -0.004037 -0.014908 -0.015549 -0.012862   
4897     1 -0.014475 -0.009675  0.005409 -0.015753 -0.016857 -0.011182   

            V7        V8        V9       V10       V11  
0     0.

In [12]:
#########################################################################
############          CLASS & FUNCTION DEFINITIONS          #############
#########################################################################

def activation_fn(z):
    z -= np.max(z)
    return np.exp(z) / np.sum(np.exp(z))
    #return (np.exp(z.T) / np.sum(np.exp(z), axis=1)).T

class SoftmaxRegressionClassifier:
    def __init__(self,
            alpha=0.001,
            epsilon=1e-5,
            epochs=100,
            batch_size=8,
            beta_decay=0.9,
            early_stop_iter=10,
            record_history=True,
            validation_set=(None,None)):
        # Gradient Descent Parameters:
        self.alpha = float(alpha)
        self.epsilon = float(epsilon)
        self.epochs = int(epochs)
        self.batch_size = int(batch_size)
        self.beta_decay = float(beta_decay)
        self.early_stop_iter = int(early_stop_iter)
        self.record_history = record_history
        if record_history:
            self.training_loss_history = []
        self.X_validation, self.y_validation = validation_set
    
    def fit(self, X, y):
        # Determine the shape of input matrix X.
        N, D = X.shape
        K = len(y.columns)
        
        # Set up the convergence check.
        previous_loss = -float('inf')
        self.converged = False
        self.stopped_early = False
        
        # Initialize the fit parameters.
        self.theta = np.zeros((D, K))
        beta = self.theta * 0
        
        for i in range(self.epochs):
            perm_fn = np.random.permutation(len(y))
            X = X.iloc[perm_fn, :]
            y = y.iloc[perm_fn]

            # Allow final batch to contain less samples (if necessary).
            batchling = (1 if N % self.batch_size else 0)

            # Divide X & y datasets into mini-batches.
            for batch_index in range(N // self.batch_size + batchling):
                mini_batch = slice(self.batch_size * batch_index, self.batch_size * (batch_index + 1))
                X_batch = X.iloc[mini_batch, :]
                y_batch = y.iloc[mini_batch]

                theta_next = self.theta + self.beta_decay * beta
                y_hat = activation_fn(X_batch @ theta_next)

                # Perform gradient descent.
                residuals = np.subtract(y_hat, y_batch)
                gradient = X_batch.T @ residuals

                beta = np.subtract((self.beta_decay * beta), (self.alpha * gradient))
                self.theta += beta
            
            # Check for convergence at end of each epoch.
            self.loss = np.mean(-1 * np.sum(y.T * (X @ self.theta)) + ((X @ self.theta).max()) \
                + (np.log(np.sum(np.exp((X @ self.theta) - (X @ self.theta).max())))))

            #self.loss = np.mean(-1 * (np.sum(y * np.log(activation_fn(X @ self.theta)+1e-6))))
            self.training_loss_history.append(self.loss)

            # Initialize early stopping mechanism.
            if self.check_validation_loss():
                self.stopped_early = True
                break
            
            if abs(previous_loss - self.loss) < self.epsilon:
                self.converged = True
                break
            else:
                previous_loss = self.loss
        
        self.iterations = i+1
    
    def predict_prob(self, X):
        return activation_fn(X @ self.theta)
    
    def predict(self, X):
        prob = self.predict_prob(X)
        return self.classlabels(prob)

    def classlabels(self, z):
        return z.idxmax(axis=1)+1
    
    def check_validation_loss(self):
        # Validation set loss history.
        if not hasattr(self, 'validation_loss_history'):
            self.validation_loss_history = []
        
        loss = np.mean(-1 * np.sum(self.y_validation.T * (self.X_validation @ self.theta)) \
            + ((self.X_validation @ self.theta).max()) + (np.log(np.sum(np.exp((self.X_validation @ self.theta) \
                - (self.X_validation @ self.theta).max())))))

        #loss = np.mean(-1 * np.sum(self.y_validation * np.log(activation_fn(self.X_validation @ self.theta)+1e-6)))
        self.validation_loss_history.append(loss)

        # Area Under Curve (AUC) & ROC history.
        if not hasattr(self, 'auc_history'):
            self.auc_history = []
        p_hat_auc = self.predict_prob(self.X_validation)
        auc = roc_auc_score(self.y_validation, p_hat_auc, average='macro', multi_class='ovo')
        self.auc_history.append(auc)

        # Implement early stopping mechanism.
        t = self.early_stop_iter

        if t and len(self.validation_loss_history) > t * 10:
            recent_best = min(self.validation_loss_history[-t:])
            previous_best = min(self.validation_loss_history[:-t])
            if recent_best > previous_best:
                return True
        return False

In [13]:
#########################################################################
############           K-FOLD CV DATA PREPARATION           #############
#########################################################################

# Shuffle/permute the X & y data (w/ equivalent indices).
perm_fn = np.random.permutation(len(y_encoded))
X_norm = X_norm.iloc[perm_fn, :]
y_encoded = y_encoded.iloc[perm_fn]

# Split into K-folds (randomly permuted).
nfolds = 5
from math import ceil
fold_size = ceil(len(y_encoded.index) / nfolds)

X_fold1, y_fold1 = X_norm.iloc[:fold_size, :], y_encoded.iloc[:fold_size, :]
X_fold2, y_fold2 = X_norm.iloc[fold_size:fold_size*2, :], y_encoded.iloc[fold_size:fold_size*2, :]
X_fold3, y_fold3 = X_norm.iloc[fold_size*2:fold_size*3, :], y_encoded.iloc[fold_size*2:fold_size*3, :]
X_fold4, y_fold4 = X_norm.iloc[fold_size*3:fold_size*4, :], y_encoded.iloc[fold_size*3:fold_size*4, :]
X_fold5, y_fold5 = X_norm.iloc[fold_size*4:, :], y_encoded.iloc[fold_size*4:, :]

# Group folds into respective train & test sets.
# Train: folds 2-3-4-5, Test: fold 1
X_train1, y_train1 = pd.concat([X_fold2, X_fold3, X_fold4, X_fold5]), pd.concat([y_fold2, y_fold3, y_fold4, y_fold5])
X_test1, y_test1 = X_fold1, y_fold1

# Train: folds 1-3-4-5, Test: fold 2
X_train2, y_train2 = pd.concat([X_fold1, X_fold3, X_fold4, X_fold5]), pd.concat([y_fold1, y_fold3, y_fold4, y_fold5])
X_test2, y_test2 = X_fold2, y_fold2

# Train: folds 1-2-4-5, Test: fold 3
X_train3, y_train3 = pd.concat([X_fold1, X_fold2, X_fold4, X_fold5]), pd.concat([y_fold1, y_fold2, y_fold4, y_fold5])
X_test3, y_test3 = X_fold3, y_fold3

# Train: folds 1-2-3-5, Test: fold 4
X_train4, y_train4 = pd.concat([X_fold1, X_fold2, X_fold3, X_fold5]), pd.concat([y_fold1, y_fold2, y_fold3, y_fold5])
X_test4, y_test4 = X_fold4, y_fold4

# Train: folds 1-2-3-4, Test: fold 5
X_train5, y_train5 = pd.concat([X_fold1, X_fold2, X_fold3, X_fold4]), pd.concat([y_fold1, y_fold2, y_fold3, y_fold4])
X_test5, y_test5 = X_fold5, y_fold5

In [ ]:
#########################################################################
############          MODEL TESTING & PERFORMANCE           #############
#########################################################################

# Initialize arrays to pull data for future calculations.

alpha = [0.1, 0.01, 0.001]

train_accuracy = []
test_accuracy = []
RMSE = []

for a in range(0, nfolds):
    train_accuracy2 = []
    train_accuracy.append(train_accuracy2)

for b in range(0, nfolds):
    test_accuracy2 = []
    test_accuracy.append(test_accuracy2)

for c in range(0, nfolds):
    RMSE2 = []
    RMSE.append(RMSE2)

# Fit & train softmax regression model using 5-fold CV.
for i in range(1, nfolds+1):
    X_train = globals()[f"X_train{i}"]
    X_test = globals()[f"X_test{i}"]
    y_train = globals()[f"y_train{i}"]
    y_test = globals()[f"y_test{i}"]

    # Loop over 'K' values.
    for j, k in enumerate(alpha):
        model = SoftmaxRegressionClassifier(alpha=k, validation_set=(X_test, y_test))
        model.fit(X_train, y_train)
        
        print("Model Stats (converged, stopped, iterations, loss):")
        print(model.converged, model.stopped_early, model.iterations, model.loss)
        print()

        # Compute predictions & test scores.
        p_hat_train = model.predict_prob(X_train)
        y_hat_train = model.predict(X_train)

        p_hat = model.predict_prob(X_test)
        y_hat = model.predict(X_test)

        # Compute & store training/test scores & RMSE.
        train_accuracy_temp = accuracy_score(model.classlabels(y_train)-1, y_hat_train)
        train_accuracy[i-1].append(train_accuracy_temp)
        
        test_accuracy_temp = accuracy_score(model.classlabels(y_test)-1, y_hat)
        test_accuracy[i-1].append(test_accuracy_temp)

        RMSE_temp = np.sqrt(np.mean(np.square(np.subtract(y_hat, model.classlabels(y_test)-1))))
        RMSE[i-1].append(RMSE_temp)

        # Print training/test scores & RMSE.
        print("Training Score (Accuracy):")
        print(train_accuracy_temp)
        print()
        
        print("Test Score (Accuracy):")
        print(test_accuracy_temp)
        print()

        print("Root Mean Square Error (RMSE):")
        print(RMSE_temp)
        print()

        # Compute & print confusion matrix.
        print("Confusion Matrix:")
        print(confusion_matrix(model.classlabels(y_test)-1, y_hat))
        print()

In [ ]:
    
    #########################################################################
    ############       GRAPHICAL PLOTS: MODEL PERFORMANCE       #############
    #########################################################################
    
    # Generate accuracy & error plots to evaluate hyperparameter choice.
    plt.plot(alpha, train_accuracy[i-1], label = 'Training Dataset Accuracy')
    plt.plot(alpha, test_accuracy[i-1], label = 'Testing Dataset Accuracy')
    plt.legend()
    plt.title(f"Test Set_K-Fold {i}")
    plt.xlabel('nfolds')
    plt.ylabel('Accuracy')
    plt.show()

    plt.plot(alpha, RMSE[i-1], label = 'RMSE')
    plt.legend()
    plt.title(f"Test Set_K-Fold {i}")
    plt.xlabel('nfolds')
    plt.ylabel('RMSE')
    plt.show()

    """
    # Compile AUC data for respective ROC curves (by class).
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    n_classes = len(y_encoded.columns)

    for j in range(n_classes):
        fpr[j], tpr[j], _ = roc_curve(y_test1.iloc[:, j], p_hat.iloc[:, j])
        roc_auc[j] = auc(fpr[j], tpr[j])

    # Print ROC_AUC values (per class).
    print("ROC AUC Values (per class):")
    from pprint import pprint
    pprint(roc_auc)
    print()

    # Plot Receiver Operating Characteristic (ROC) curves (by class).
    for k in range(n_classes):
        plt.figure(figsize=(10, 6))
        plt.step(fpr[k], tpr[k], color='black')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f"ROC Curve (Class {k+1})")
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
        plt.text(0.4, 0.6, 'AUC: {:.4f}'.format(roc_auc[k]))
        plt.show()

    # Plot the training & validation loss curves (by class).
    plt.figure(figsize=(16,6))
    plt.ylim(0, 40)
    plt.xscale('log')
    plt.plot(range(1, model.iterations+1), model.training_loss_history, label='Training Loss')
    plt.plot(range(1, model.iterations+1), model.validation_loss_history, label='Validation Loss')
    plt.xlabel("Iterations")
    plt.ylabel("Loss Data")
    plt.title("Training & Validation Loss Curves")
    plt.legend()
    plt.grid(True, 'both')
    plt.plot(
        [model.training_loss_history.index(min(model.training_loss_history))], [min(model.training_loss_history)], 
        marker='o', markersize=7, markerfacecolor='none', color="blue")
    plt.plot(
        [model.validation_loss_history.index(min(model.validation_loss_history))], [min(model.validation_loss_history)], 
        marker='o', markersize=7, markerfacecolor='none', color="orange")
    plt.show()
"""

In [ ]:

#########################################################################
########       K-FOLD CV MODEL PERFORMANCE AVERAGE METRICS       ########
#########################################################################

# Zip train/test accuracies & RMSE to K-hyperparameter arrays.
k_train_accuracy = list(map(list, zip(*train_accuracy)))
k_test_accuracy = list(map(list, zip(*test_accuracy)))
k_RMSE = list(map(list, zip(*RMSE)))

# Initialize arrays to store K-hyperparameter metrics.
mean_k_train_accuracy = []
mean_k_test_accuracy = []
mean_k_RMSE = []

# Compute metrics based on each hyperparameter.
for m in range(len(alpha)):
    mean_k_train_accuracy_temp = np.mean(k_train_accuracy[m])
    mean_k_train_accuracy.append(mean_k_train_accuracy_temp)

    mean_k_test_accuracy_temp = np.mean(k_test_accuracy[m])
    mean_k_test_accuracy.append(mean_k_test_accuracy_temp)

    mean_k_RMSE_temp = np.mean(k_RMSE[m])
    mean_k_RMSE.append(mean_k_RMSE_temp)

# Print region-based summary tables (training/testing accuracy & RMSE).
train_accuracy_table = pd.DataFrame(train_accuracy)
train_accuracy_table.loc['Mean (Digits)'] = train_accuracy_table.mean()
train_accuracy_table['Mean (nFold)'] = train_accuracy_table.mean(numeric_only=True, axis=1)
print("Table: Training Accuracy (5-Fold CV):")
print(train_accuracy_table)
print()

test_accuracy_table = pd.DataFrame(test_accuracy)
test_accuracy_table.loc['Mean (Digits)'] = test_accuracy_table.mean()
test_accuracy_table['Mean (nFold)'] = test_accuracy_table.mean(numeric_only=True, axis=1)
print("Table: Testing Accuracy (5-Fold CV):")
print(test_accuracy_table)
print()

RMSE_table = pd.DataFrame(RMSE)
RMSE_table.loc['Mean (Digits)'] = RMSE_table.mean()
RMSE_table['Mean (nFold)'] = RMSE_table.mean(numeric_only=True, axis=1)
print("Table: RMSE (5-Fold CV):")
print(RMSE_table)
print()

train_accuracy_table.to_csv('wine-softmax_train_accuracy_table.csv')
test_accuracy_table.to_csv('wine-softmax_test_accuracy_table.csv')
RMSE_table.to_csv('wine-softmax_RMSE_table.csv')